# NN - Based on `nn` by Professor Pooyan Jamshidi - By Walter Pach

In [ ]:
import numpy as np
import random
import csv

In [ ]:
# Credit to Professor Pooyan Jamshidi for these two functions
def linear(x, weights, bias):
    return np.dot(x, weights) + bias

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [ ]:
# An array of files to read training data from
training_files = ["and.csv", "nand.csv", "nor.csv", "or.csv", "xor.csv"]

# Read CSV in as an array of values.
# Format: element0: x1, element1: x2, element3: classification
def create_training_set(filename):
    rows = []
    
    # Opens the file
    with open(filename) as fcsv:
        # Reads as CSV
        fcsv_reader = csv.reader(fcsv)

        # Ignore first row.
        next(fcsv_reader)        
        for row in fcsv_reader:
            # Append each row to the array
            rows.append([int(x) for x in row])
    return rows

The code below creates a neural network from the logical gate that you specify. It then uses the two feature vectors 
and provides with a classification, infering the logical value of the gate.

**Input Features:**
1. `x1`
2. `x2`

**Output Features:**
1. `classification label`

In [ ]:
# There are two input features, x1 and x2
num_input_features = 2
# There is one output feature, classification
num_output_features = 1
# Random value for the number of hidden neurons
num_hidden_neurons = 2
# Gate to do!
logic_gate = 'or'

In [ ]:
def blank_neural_network():
    # Create weights from the inputs to the hidden layer.
    weights1 = np.zeros((num_hidden_neurons, num_input_features))
    # Create weights from the hidden layer to the output feature.
    weights2 = np.zeros((num_output_features, num_hidden_neurons))
    # Biases on the hidden neurons
    biases1 = np.zeros((num_hidden_neurons, 1))
    # Biases on the output feature
    biases2 = np.zeros((num_output_features, 1))
    
    return (weights1, weights2, biases1, biases2)

# Creates a step for getting the y-star and classification
def step(x):
    if x >= 0.5:
        return 1
    else:
        return 0
    
# Calculated the classification based on the parameters
def classify(input_features, weights1, weights2, biases1, biases2):
    # Activation of layer 1 calculation
    activation1 = linear(weights1, input_features, biases1)
    # Sigmoid of that activation, to be passed to the next layer
    sigmoid1 = sigmoid(activation1)
    # Activation of layer 2 calculation
    activation2 = linear(weights2, sigmoid1, biases2)
    # Sigmoid of that activation
    sigmoid2 = sigmoid(activation2)
    
    # Sum of all resulting inputs to the output layer. This is the activation of the output neuron.
    return np.sum(sigmoid2)
    
def train(epochs):
    # Creates a blank network
    weights1, weights2, biases1, biases2 = blank_neural_network()

    # Fetches the test cases for the logic gate specified
    test_cases = create_training_set(f"{logic_gate}.csv")
    
    # Loops over each epoch specified
    for i in range(epochs):
        # Fetches a line of the test cases
        test_case = test_cases[i % len(test_cases)]
        # Chooses the first two elements, the inputs
        input_features = [test_case[0], test_case[1]]
        # Calculates the classification from the inputs
        classification = classify(input_features, weights1, weights2, biases1, biases2)
        # Apply the step function to that classification
        y = step(classification)
        # y-star is the "correct" value, according to the slides.
        y_star = step(test_case[2])
        
        
        # If the two are equal, then the classification was correct
        if y == y_star:
            print(f"Predicted correctly! (x1, x2, y, y-star): ({test_case[0]}, {test_case[1]}, {y}, {y_star})")
            continue
        # If this condition is true, then the output neuron did not activate when it should have
        elif y == 0 and y_star == 1:
            # Multiplies the input features by y_star
            y_star_f = np.multiply(1, input_features)
            # Adds this to the weights
            weights1 = np.add(weights1, y_star_f)
            weights2 = np.add(weights2, y_star_f)
            # Debug print
            print(f"Incorrect prediction:")
            print(f"  x1: {test_case[0]}")
            print(f"  x2: {test_case[1]}")
            print(f"  y:  {y}")
            print(f"  y-star: {y_star}")
            print(f"  classification: {classification}")
        # If this condition is true, then the output neuron activated when it should not have.
        elif y == 1 and y_star == 0:
            # Multiplies the input features by y_star
            y_star_f = np.multiply(1, input_features)
            # Subtracts since it is wrong
            weights1 = np.subtract(weights1, y_star_f)
            weights2 = np.subtract(weights2, y_star_f)
            # Debug print
            print(f"{y_star_f}")
            print(f"Incorrect prediction:")
            print(f"  x1: {test_case[0]}")
            print(f"  x2: {test_case[1]}")
            print(f"  y:  {y}")
            print(f"  y-star: {y_star}")
            print(f"  classification: {classification}")
    
# Trains for 3 epochs
train(3)